### Read the dataset

In [2]:
with open('data/input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

### Data exploration and preprocessing of dataset

In [6]:
print("Length of the text is: {}".format(len(text)))

Length of the text is: 1115394


In [7]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [10]:
chars = sorted(list(set(text)))
print("".join(chars))


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


In [11]:
vocab_size = len(chars)
print("Vocabulary size is: {}".format(vocab_size))

Vocabulary size is: 65


### Tokenization

There are different ways to tokenize the text. Google uses SentencePiece tokenizer mechanism [Link](https://github.com/google/sentencepiece) that uses subwords to tokenize the sentences. OpenAI which created chatCPT uses tiktoker library developed by them [Link](https://github.com/openai/tiktoken) that uses a BPE (byte-pair encoding) to tokenize.

For this work we will be using character level encoding (converting the text to their ASCII equivalent).

There is a tradeoff between the encoding numbers (below it is the ASCII codes) and the sequence length generated (a code for every character below).

In [15]:
stoi = {ch: i for i,ch in enumerate(chars)}
itos = {i: ch for i,ch in enumerate(chars)}

encode = lambda s: [stoi[c] for c in s]
decode = lambda l: "".join([itos[i] for i in l])

print("Encoding for the string \"hi there\" is: {}".format(encode("hii there")))
print("Decoding for the string \"hi there\" is: {}".format(decode(encode("hii there"))))

Encoding for the string "hi there" is: [46, 47, 47, 1, 58, 46, 43, 56, 43]
Decoding for the string "hi there" is: hii there


### Encoding the entire data and using storing them as tensors

In [16]:
import torch

In [17]:
data = torch.tensor(encode(text), dtype = torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

### Train test split

In [19]:
train_data_len = int(0.9 * len(data))

In [20]:
train_data = data[:train_data_len]
val_data = data[train_data_len:]

### Batch size and batches for training

In [21]:
# this is the chunk of data that will be fed to model at once, this might be known as context-length in other cases
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [24]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

for i,c in enumerate(x):
    print("When the context is {}, the most probable next data is {}".format(x[:i+1], y[i]))

When the context is tensor([18]), the most probable next data is 47
When the context is tensor([18, 47]), the most probable next data is 56
When the context is tensor([18, 47, 56]), the most probable next data is 57
When the context is tensor([18, 47, 56, 57]), the most probable next data is 58
When the context is tensor([18, 47, 56, 57, 58]), the most probable next data is 1
When the context is tensor([18, 47, 56, 57, 58,  1]), the most probable next data is 15
When the context is tensor([18, 47, 56, 57, 58,  1, 15]), the most probable next data is 47
When the context is tensor([18, 47, 56, 57, 58,  1, 15, 47]), the most probable next data is 58


In [26]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parellel?
block_size = 8 # What is the maximum length allowed for prediction?

def get_batch(split):
    # generate a small batch of data of input x and outputs y
    data = train_data if split=="train" else val_data
    ix = torch.randint(len(data)-block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    
    return x, y

In [27]:
xb, yb = get_batch("train")
print(f"Inputs are {xb} with shape {xb.shape}")
print(f"Outputs are {yb} with shape {yb.shape}")

Inputs are tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]]) with shape torch.Size([4, 8])
Outputs are tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]]) with shape torch.Size([4, 8])


In [28]:
for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"When input is {context.tolist()} then most probable output is {target.tolist()}")

When input is [24] then most probable output is 43
When input is [24, 43] then most probable output is 58
When input is [24, 43, 58] then most probable output is 5
When input is [24, 43, 58, 5] then most probable output is 57
When input is [24, 43, 58, 5, 57] then most probable output is 1
When input is [24, 43, 58, 5, 57, 1] then most probable output is 46
When input is [24, 43, 58, 5, 57, 1, 46] then most probable output is 43
When input is [24, 43, 58, 5, 57, 1, 46, 43] then most probable output is 39
When input is [44] then most probable output is 53
When input is [44, 53] then most probable output is 56
When input is [44, 53, 56] then most probable output is 1
When input is [44, 53, 56, 1] then most probable output is 58
When input is [44, 53, 56, 1, 58] then most probable output is 46
When input is [44, 53, 56, 1, 58, 46] then most probable output is 39
When input is [44, 53, 56, 1, 58, 46, 39] then most probable output is 58
When input is [44, 53, 56, 1, 58, 46, 39, 58] then mos